# Attention Pooling: Nadaraya-Watson Kernel Regression
:label:`sec_nadaraya-watson`

Now you know the major components of attention mechanisms under the framework in :numref:`fig_qkv`.
To recapitulate,
the interactions between
queries (volitional cues) and keys (nonvolitional cues)
result in *attention pooling*.
The attention pooling selectively aggregates values (sensory inputs) to produce the output.
In this section,
we will describe attention pooling in greater detail
to give you a high-level view of
how attention mechanisms work in practice.
Specifically,
the Nadaraya-Watson kernel regression model
proposed in 1964
is a simple yet complete example
for demonstrating machine learning with attention mechanisms.


In [106]:
%use @file[../djl-pytorch.json]
%use lets-plot
@file:DependsOn("../D2J-1.0-SNAPSHOT.jar")
import ai.djl.Model
import ai.djl.ndarray.NDArray
import ai.djl.ndarray.NDList
import ai.djl.ndarray.NDManager
import ai.djl.ndarray.index.NDIndex
import ai.djl.ndarray.types.DataType
import ai.djl.ndarray.types.Shape
import ai.djl.nn.AbstractBlock
import ai.djl.nn.Parameter
import ai.djl.training.DefaultTrainingConfig
import ai.djl.training.GradientCollector
import ai.djl.training.ParameterStore
import ai.djl.training.Trainer
import ai.djl.training.initializer.UniformInitializer
import ai.djl.training.loss.Loss
import ai.djl.training.optimizer.Optimizer
import ai.djl.training.tracker.Tracker
import ai.djl.util.PairList


In [107]:
val manager = NDManager.newBaseManager()


## Generating the Dataset

To keep things simple,
let us consider the following regression problem:
given a dataset of input-output pairs $\{(x_1, y_1), \ldots, (x_n, y_n)\}$,
how to learn $f$ to predict the output $\hat{y} = f(x)$ for any new input $x$?

Here we generate an artificial dataset according to the following nonlinear function with the noise term $\epsilon$:

$$y_i = 2\sin(x_i) + x_i^{0.8} + \epsilon,$$

where $\epsilon$ obeys a normal distribution with zero mean and standard deviation 0.5.
Both 50 training examples and 50 testing examples
are generated.
To better visualize the pattern of attention later, the training inputs are sorted.


In [108]:
val nTrain = 50 // No. of training examples
val xTrain = manager.randomUniform(0f, 1f, Shape(nTrain.toLong())).mul(5).sort() // Training inputs


In [109]:
val f: (NDArray) -> NDArray = { x -> x.sin().mul(2).add(x.pow(0.8)) }
val yTrain =
    f(xTrain).add(
        manager.randomNormal(
            0f,
            0.5f,
            Shape(nTrain.toLong()),
            DataType.FLOAT32
        )
    ) // Training outputs
val xTest = manager.arange(0f, 5f, 0.1f) // Testing examples
val yTruth = f(xTest) // Ground-truth outputs for the testing examples
val nTest = xTest.shape[0].toInt() // No. of testing examples
println(nTest)


50


The following function plots all the training examples (represented by circles),
the ground-truth data generation function `f` without the noise term (labeled by "Truth"), and the learned prediction function (labeled by "Pred").


In [110]:
fun plot(
    yHat: NDArray,
    trace1Name: String,
    trace2Name: String,
    xLabel: String,
    yLabel: String,
    width: Int,
    height: Int
): Any {
    val data = mapOf(
        "x" to (xTest.toFloatArray() + xTest.toFloatArray()),
        "y" to (yTruth.toFloatArray() + yHat.toFloatArray()),
        "label" to (Array(nTest) { trace1Name } + Array(nTest) { trace2Name })
    )
    var plot = letsPlot(data)
    plot += geomLine(size = 2) { x = "x"; y = "y"; color = "label" }
    plot += geomPoint(size = 3) { x = xTrain.toFloatArray(); y = yTrain.toFloatArray() }
    plot += xlab(xLabel) + ylab(yLabel)
    return plot + ggsize(width, height)
}


## Average Pooling

We begin with perhaps the world's "dumbest" estimator for this regression problem:
using average pooling to average over all the training outputs:

$$f(x) = \frac{1}{n}\sum_{i=1}^n y_i,$$
:eqlabel:`eq_avg-pooling`

which is plotted below. As we can see, this estimator is indeed not so smart.


In [111]:
var yHat = yTrain.mean().tile(nTest.toLong())
plot(yHat, "Truth", "Pred", "x", "y", 700, 500)


<path d="M25.865392601084185 400.8782798224489 L25.865392601084185 400.8782798224489 L36.42269565540954 372.1087810000674 L46.9799987097349 346.79555496296905 L57.537302550640625 322.74303050009826 L68.09460481838562 299.72396695825523 L78.65190708613062 277.72137227898895 L89.20921250019707 256.78626572824885 L99.76651162162062 236.9962391063333 L110.32381703568706 218.4383641447135 L120.8811224497535 201.2005743867856 L131.43842157117706 185.36642728650523 L141.99572698524352 171.01178144463296 L152.55303239930996 158.20257504910577 L163.1103378133764 146.99287243467427 L173.66763064215706 137.4238107572167 L184.2249360562235 129.52250836530078 L194.78224147028993 123.30147110752449 L205.3395468843564 118.75813269948958 L215.89685229842283 115.87462490728791 L226.4541451272035 114.61789245575858 L237.01145054126994 114.9396547257353 L247.56876854062216 116.77703774945832 L258.12606136940286 120.0527848757111 L268.6833541981835 124.67625264137854 L279.24067219753573 130.54408106961006 L289.7979650263164 137.54107463312096 L300.3552830256686 145.54160030464817 L310.91257585444924 154.4104110661228 L321.46986868322995 164.00429292701529 L332.02718668258217 174.17330976378256 L342.5844795113628 184.76214391619533 L353.14179751071504 195.61193471944458 L363.6990903394957 206.56152334358833 L374.2563831682764 217.4492338715298 L384.8137011676286 228.114443711858 L395.37099399640925 238.399019952056 L405.9283119957615 248.14914831491893 L416.4856048245421 257.216549270937 L427.0428976533228 265.46021123783294 L437.60021565267505 272.74765457138534 L448.1575084814557 278.9561668291875 L458.71480131023634 283.97421039679114 L469.27214448016014 287.70242793495225 L479.8294373089408 290.0545616456841 L490.3867301377215 290.9583821139983 L500.94402296650213 290.3564352115735 L511.5013157952828 288.20648257840526 L522.0586589652066 284.4820665884014 L532.6159517939873 279.17265398470306 L543.1732446227679 272.28375078794113 " fill="none" stroke-width="4.4" stroke="rgb(228,26,28)" stroke-opacity="1.0">
 
 
 
 <path d="M25.865392601084185 200.16467645314816 L25.865392601084185 200.16467645314816 L36.42269565540954 200.16467645314816 L46.9799987097349 200.16467645314816 L57.537302550640625 200.16467645314816 L68.09460481838562 200.16467645314816 L78.65190708613062 200.16467645314816 L89.20921250019707 200.16467645314816 L99.76651162162062 200.16467645314816 L110.32381703568706 200.16467645314816 L120.8811224497535 200.16467645314816 L131.43842157117706 200.16467645314816 L141.99572698524352 200.16467645314816 L152.55303239930996 200.16467645314816 L163.1103378133764 200.16467645314816 L173.66763064215706 200.16467645314816 L184.2249360562235 200.16467645314816 L194.78224147028993 200.16467645314816 L205.3395468843564 200.16467645314816 L215.89685229842283 200.16467645314816 L226.4541451272035 200.16467645314816 L237.01145054126994 200.16467645314816 L247.56876854062216 200.16467645314816 L258.12606136940286 200.16467645314816 L268.6833541981835 200.16467645314816 L279.24067219753573 200.16467645314816 L289.7979650263164 200.16467645314816 L300.3552830256686 200.16467645314816 L310.91257585444924 200.16467645314816 L321.46986868322995 200.16467645314816 L332.02718668258217 200.16467645314816 L342.5844795113628 200.16467645314816 L353.14179751071504 200.16467645314816 L363.6990903394957 200.16467645314816 L374.2563831682764 200.16467645314816 L384.8137011676286 200.16467645314816 L395.37099399640925 200.16467645314816 L405.9283119957615 200.16467645314816 L416.4856048245421 200.16467645314816 L427.0428976533228 200.16467645314816 L437.60021565267505 200.16467645314816 L448.1575084814557 200.16467645314816 L458.71480131023634 200.16467645314816 L469.27214448016014 200.16467645314816 L479.8294373089408 200.16467645314816 L490.3867301377215 200.16467645314816 L500.94402296650213 200.16467645314816 L511.5013157952828 200.16467645314816 L522.0586589652066 200.16467645314816 L532.6159517939873 200.16467645314816 L543.1732446227679 200.1646764531481

## Nonparametric Attention Pooling

Obviously,
average pooling omits the inputs $x_i$.
A better idea was proposed
by Nadaraya :cite:`Nadaraya.1964`
and Watson :cite:`Watson.1964`
to weigh the outputs $y_i$ according to their input locations:

$$f(x) = \sum_{i=1}^n \frac{K(x - x_i)}{\sum_{j=1}^n K(x - x_j)} y_i,$$
:eqlabel:`eq_nadaraya-watson`

where $K$ is a *kernel*.
The estimator in :eqref:`eq_nadaraya-watson`
is called *Nadaraya-Watson kernel regression*.
Here we will not dive into details of kernels.
Recall the framework of attention mechanisms in :numref:`fig_qkv`.
From the perspective of attention,
we can rewrite :eqref:`eq_nadaraya-watson`
in a more generalized form of *attention pooling*:

$$f(x) = \sum_{i=1}^n \alpha(x, x_i) y_i,$$
:eqlabel:`eq_attn-pooling`


where $x$ is the query and $(x_i, y_i)$ is the key-value pair.
Comparing :eqref:`eq_attn-pooling` and :eqref:`eq_avg-pooling`,
the attention pooling here
is a weighted average of values $y_i$.
The *attention weight* $\alpha(x, x_i)$
in :eqref:`eq_attn-pooling`
is assigned to the corresponding value $y_i$
based on the interaction
between the query $x$ and the key $x_i$
modeled by $\alpha$.
For any query, its attention weights over all the key-value pairs are a valid probability distribution: they are non-negative and sum up to one.

To gain intuitions of attention pooling,
just consider a *Gaussian kernel* defined as

$$
K(u) = \frac{1}{\sqrt{2\pi}} \exp(-\frac{u^2}{2}).
$$


Plugging the Gaussian kernel into
:eqref:`eq_attn-pooling` and
:eqref:`eq_nadaraya-watson` gives

$$\begin{aligned} f(x) &=\sum_{i=1}^n \alpha(x, x_i) y_i\\ &= \sum_{i=1}^n \frac{\exp\left(-\frac{1}{2}(x - x_i)^2\right)}{\sum_{j=1}^n \exp\left(-\frac{1}{2}(x - x_j)^2\right)} y_i \\&= \sum_{i=1}^n \mathrm{softmax}\left(-\frac{1}{2}(x - x_i)^2\right) y_i. \end{aligned}$$
:eqlabel:`eq_nadaraya-watson-gaussian`

In :eqref:`eq_nadaraya-watson-gaussian`,
a key $x_i$ that is closer to the given query $x$ will get
*more attention* via a *larger attention weight* assigned to the key's corresponding value $y_i$.

Notably, Nadaraya-Watson kernel regression is a nonparametric model;
thus :eqref:`eq_nadaraya-watson-gaussian`
is an example of *nonparametric attention pooling*.
In the following, we plot the prediction based on this
nonparametric attention model.
The predicted line is smooth and closer to the ground-truth than that produced by average pooling.


In [112]:
// Shape of `xRepeat`: (`nTest`, `nTrain`), where each row contains the
// same testing inputs (i.e., same queries)
val xRepeat = xTest.repeat(nTrain.toLong()).reshape(Shape(-1, nTrain.toLong()))
// Note that `xTrain` contains the keys. Shape of `attentionWeights`:
// (`nTest`, `nTrain`), where each row contains attention weights to be
// assigned among the values (`yTrain`) given each query
val attentionWeights = xRepeat.sub(xTrain).pow(2).div(2).mul(-1).softmax(-1)
// Each element of `yHat` is weighted average of values, where weights are
// attention weights
yHat = attentionWeights.matMul(yTrain.reshape(Shape(nTrain.toLong(), 1))).reshape(Shape(-1))
plot(yHat, "Truth", "Pred", "x", "y", 700, 500)


<path d="M25.865392601084185 400.8782798224489 L25.865392601084185 400.8782798224489 L36.42269565540954 372.1087810000674 L46.9799987097349 346.79555496296905 L57.537302550640625 322.74303050009826 L68.09460481838562 299.72396695825523 L78.65190708613062 277.72137227898895 L89.20921250019707 256.78626572824885 L99.76651162162062 236.9962391063333 L110.32381703568706 218.4383641447135 L120.8811224497535 201.2005743867856 L131.43842157117706 185.36642728650523 L141.99572698524352 171.01178144463296 L152.55303239930996 158.20257504910577 L163.1103378133764 146.99287243467427 L173.66763064215706 137.4238107572167 L184.2249360562235 129.52250836530078 L194.78224147028993 123.30147110752449 L205.3395468843564 118.75813269948958 L215.89685229842283 115.87462490728791 L226.4541451272035 114.61789245575858 L237.01145054126994 114.9396547257353 L247.56876854062216 116.77703774945832 L258.12606136940286 120.0527848757111 L268.6833541981835 124.67625264137854 L279.24067219753573 130.54408106961006 L289.7979650263164 137.54107463312096 L300.3552830256686 145.54160030464817 L310.91257585444924 154.4104110661228 L321.46986868322995 164.00429292701529 L332.02718668258217 174.17330976378256 L342.5844795113628 184.76214391619533 L353.14179751071504 195.61193471944458 L363.6990903394957 206.56152334358833 L374.2563831682764 217.4492338715298 L384.8137011676286 228.114443711858 L395.37099399640925 238.399019952056 L405.9283119957615 248.14914831491893 L416.4856048245421 257.216549270937 L427.0428976533228 265.46021123783294 L437.60021565267505 272.74765457138534 L448.1575084814557 278.9561668291875 L458.71480131023634 283.97421039679114 L469.27214448016014 287.70242793495225 L479.8294373089408 290.0545616456841 L490.3867301377215 290.9583821139983 L500.94402296650213 290.3564352115735 L511.5013157952828 288.20648257840526 L522.0586589652066 284.4820665884014 L532.6159517939873 279.17265398470306 L543.1732446227679 272.28375078794113 " fill="none" stroke-width="4.4" stroke="rgb(228,26,28)" stroke-opacity="1.0">
 
 
 
 <path d="M25.865392601084185 235.775606998766 L25.865392601084185 235.775606998766 L36.42269565540954 231.05609508192106 L46.9799987097349 226.1656954360718 L57.537302550640625 221.11490301532484 L68.09460481838562 215.9208583012962 L78.65190708613062 210.60818996365964 L89.20921250019707 205.2093021307886 L99.76651162162062 199.76485317415853 L110.32381703568706 194.3233726808251 L120.8811224497535 188.94084012314957 L131.43842157117706 183.67938256522362 L141.99572698524352 178.60560849314805 L152.55303239930996 173.78896079668752 L163.1103378133764 169.29915048487214 L173.66763064215706 165.20362870435142 L184.2249360562235 161.5657482072879 L194.78224147028993 158.4421970669588 L205.3395468843564 155.88154317317205 L215.89685229842283 153.92274042492951 L226.4541451272035 152.59434352400712 L237.01145054126994 151.91431646119332 L247.56876854062216 151.89010912179384 L258.12606136940286 152.51879134526428 L268.6833541981835 153.7879721912633 L279.24067219753573 155.6763936323119 L289.7979650263164 158.1546774574617 L300.3552830256686 161.18624453834798 L310.91257585444924 164.72796597597736 L321.46986868322995 168.73131218329422 L332.02718668258217 173.14308063747285 L342.5844795113628 177.90627684321822 L353.14179751071504 182.9613783235893 L363.6990903394957 188.24723473467577 L374.2563831682764 193.70250421880638 L384.8137011676286 199.26672588161074 L395.37099399640925 204.8816220855947 L405.9283119957615 210.49224753270676 L416.4856048245421 216.04792768176733 L427.0428976533228 221.5031014090174 L437.60021565267505 226.8180487604107 L448.1575084814557 231.9589292543658 L458.71480131023634 236.8983755744256 L469.27214448016014 241.61493817935 L479.8294373089408 246.09298954623557 L490.3867301377215 250.32232199161717 L500.94402296650213 254.29756355449678 L511.5013157952828 258.01742151698704 L522.0586589652066 261.48428980110157 L532.6159517939873 264.7032722485727 L543.1732446227679 267.6819719557151 " fill="none" stroke-w

Now let us take a look at the attention weights.
Here testing inputs are queries while training inputs are keys.
Since both inputs are sorted,
we can see that the closer the query-key pair is,
the higher attention weight is in the attention pooling.


In [113]:
fun plotHeatmap(att: NDArray, width: Int, height: Int): Any {
    val seriesX = mutableListOf<Long>()
    val seriesY = mutableListOf<Long>()
    val seriesW = mutableListOf<Float>()
    for (i in 0 until att.shape[0]) {
        val row = att.get(i)
        for (j in 0 until row.shape[0]) {
            seriesX.add(j)
            seriesY.add(i)
            seriesW.add(row.get(j).getFloat())
        }
    }
    val data = mapOf("x" to seriesX, "y" to seriesY)
    var plot = letsPlot(data)
    plot += geomBin2D(drop = false, binWidth = kotlin.Pair(1, 1), position = positionIdentity) {
        x = "x"; y = "y"; weight = seriesW
    }
    plot += scaleFillGradient(low = "blue", high = "red")
    return plot + ggsize(width, height)
}

plotHeatmap(attentionWeights, 500, 700)


<rect fill="#9f00be" stroke="#ffffff" stroke-width="0.0" x="109.6

## Parametric Attention Pooling

Nonparametric Nadaraya-Watson kernel regression
enjoys the *consistency* benefit:
given enough data this model converges to the optimal solution.
Nonetheless,
we can easily integrate learnable parameters into attention pooling.

As an example, slightly different from :eqref:`eq_nadaraya-watson-gaussian`,
in the following
the distance between the query $x$ and the key $x_i$
is multiplied a learnable parameter $w$:


$$\begin{aligned}f(x) &= \sum_{i=1}^n \alpha(x, x_i) y_i \\&= \sum_{i=1}^n \frac{\exp\left(-\frac{1}{2}((x - x_i)w)^2\right)}{\sum_{j=1}^n \exp\left(-\frac{1}{2}((x - x_i)w)^2\right)} y_i \\&= \sum_{i=1}^n \mathrm{softmax}\left(-\frac{1}{2}((x - x_i)w)^2\right) y_i.\end{aligned}$$
:eqlabel:`eq_nadaraya-watson-gaussian-para`

In the rest of the section,
we will train this model by learning the parameter of
the attention pooling in :eqref:`eq_nadaraya-watson-gaussian-para`.


### Batch Matrix Multiplication
:label:`subsec_batch_dot`

To more efficiently compute attention
for minibatches,
we can leverage batch matrix multiplication utilities
provided by deep learning frameworks.


Suppose that the first minibatch contains $n$ matrices $\mathbf{X}_1, \ldots, \mathbf{X}_n$ of shape $a\times b$, and the second minibatch contains $n$ matrices $\mathbf{Y}_1, \ldots, \mathbf{Y}_n$ of shape $b\times c$. Their batch matrix multiplication
results in
$n$ matrices $\mathbf{X}_1\mathbf{Y}_1, \ldots, \mathbf{X}_n\mathbf{Y}_n$ of shape $a\times c$. Therefore, given two tensors of shape ($n$, $a$, $b$) and ($n$, $b$, $c$), the shape of their batch matrix multiplication output is ($n$, $a$, $c$).


In [114]:
val X = manager.ones(Shape(2, 1, 4))
val Y = manager.ones(Shape(2, 4, 6))

X.matMul(Y).shape


(2, 1, 6)

In the context of attention mechanisms, we can use minibatch matrix multiplication to compute weighted averages of values in a minibatch.


In [115]:
val weights = manager.ones(Shape(2, 10)).mul(0.1)
val values = manager.arange(20f).reshape(Shape(2, 10))
weights.expandDims(1).matMul(values.expandDims(-1))


ND: (2, 1, 1) gpu(0) float32
Check the "Development Guideline"->Debug to enable array display.


### Defining the Model

Using minibatch matrix multiplication,
below we define the parametric version
of Nadaraya-Watson kernel regression
based on the parametric attention pooling in
:eqref:`eq_nadaraya-watson-gaussian-para`.


In [116]:
class NWKernelRegression : AbstractBlock() {
    private val w: Parameter
    var attentionWeights: NDArray? = null

    init {
        w = Parameter.builder()
            .optShape(Shape(1))
            .setName("w")
            .optInitializer(UniformInitializer())
            .build()
        addParameter(w)
    }

    override fun forwardInternal(
        parameterStore: ParameterStore,
        inputs: NDList,
        training: Boolean,
        params: PairList<String, Any>?
    ): NDList {
        // Shape of the output `queries` and `attentionWeights`:
        // (no. of queries, no. of key-value pairs)
        var queries = inputs[0]
        val keys = inputs[1]
        val values = inputs[2]
        queries = queries.repeat(keys.shape[1]).reshape(Shape(-1, keys.shape[1]))

        attentionWeights = queries.sub(keys).mul(w.array).pow(2).div(2).mul(-1).softmax(-1)
        // Shape of `values`: (no. of queries, no. of key-value pairs)
        return NDList(attentionWeights!!.mul(values).sum(intArrayOf(1)))
    }

    override fun getOutputShapes(inputShapes: Array<Shape>): Array<Shape> {
        throw UnsupportedOperationException("Not implemented")
    }
}


### Training

In the following, we transform the training dataset
to keys and values to train the attention model.
In the parametric attention pooling,
any training input takes key-value pairs from all the training examples except for itself to predict its output.


In [117]:
// Shape of `xTile`: (`nTrain`, `nTrain`), where each column contains the
// same training inputs
val xTile = xTrain.tile(longArrayOf(nTrain.toLong(), 1L))
// Shape of `yTile`: (`nTrain`, `nTrain`), where each column contains the
// same training outputs
val yTile = yTrain.tile(longArrayOf(nTrain.toLong(), 1L))
// Shape of `keys`: (`nTrain`, `nTrain` - 1)
var keys =
    xTile.get(NDIndex().addBooleanIndex(manager.eye(nTrain).mul(-1).add(1).toType(DataType.BOOLEAN, false))).reshape(Shape(nTrain.toLong(), -1))
// Shape of `values`: (`nTrain`, `nTrain` - 1)
var valuesKV =
    yTile.get(NDIndex().addBooleanIndex(manager.eye(nTrain).mul(-1).add(1).toType(DataType.BOOLEAN, false))).reshape(Shape(nTrain.toLong(), -1))


Using the squared loss and stochastic gradient descent,
we train the parametric attention model.


In [118]:
val net = NWKernelRegression()
val loss = Loss.l2Loss()
val lrt = Tracker.fixed(0.5f * nTrain) // scale for SGD
val sgd = Optimizer.sgd().setLearningRateTracker(lrt).build()
val config =
    DefaultTrainingConfig(loss)
        .optOptimizer(sgd) // Optimizer (loss function)
        val model = Model.newInstance("")
model.block = net

val trainer: Trainer = model.newTrainer(config)
//val animator = Animator()
val ps = ParameterStore(manager, false)

for (epoch in 0 until 5) {
    trainer.newGradientCollector().use { gc ->
        val result = net.forward(ps, NDList(xTrain, keys, valuesKV), true).singletonOrThrow()
        val l = trainer.getLoss().evaluate(NDList(yTrain), NDList(result))
        gc.backward(l)
//        animator.add(epoch + 1, l.getFloat(), "Loss")
//        animator.show()
    }
    trainer.step()
}


After training the parametric attention model,
we can plot its prediction.
Trying to fit the training dataset with noise,
the predicted line is less smooth
than its nonparametric counterpart that was plotted earlier.


In [119]:
// Shape of `keys`: (`nTest`, `nTrain`), where each column contains the same
// training inputs (i.e., same keys)
keys = xTrain.tile(longArrayOf(nTest.toLong(), 1L))

// Shape of `values`: (`nTest`, `nTrain`)
valuesKV = yTrain.tile(longArrayOf(nTest.toLong(), 1L))
yHat = net.forward(ps, NDList(xTest, keys, valuesKV), true).singletonOrThrow()
plot(yHat, "Truth", "Pred", "x", "y", 700, 500)


<path d="M25.865392601084185 400.8782798224489 L25.865392601084185 400.8782798224489 L36.42269565540954 372.1087810000674 L46.9799987097349 346.79555496296905 L57.537302550640625 322.74303050009826 L68.09460481838562 299.72396695825523 L78.65190708613062 277.72137227898895 L89.20921250019707 256.78626572824885 L99.76651162162062 236.9962391063333 L110.32381703568706 218.4383641447135 L120.8811224497535 201.2005743867856 L131.43842157117706 185.36642728650523 L141.99572698524352 171.01178144463296 L152.55303239930996 158.20257504910577 L163.1103378133764 146.99287243467427 L173.66763064215706 137.4238107572167 L184.2249360562235 129.52250836530078 L194.78224147028993 123.30147110752449 L205.3395468843564 118.75813269948958 L215.89685229842283 115.87462490728791 L226.4541451272035 114.61789245575858 L237.01145054126994 114.9396547257353 L247.56876854062216 116.77703774945832 L258.12606136940286 120.0527848757111 L268.6833541981835 124.67625264137854 L279.24067219753573 130.54408106961006 L289.7979650263164 137.54107463312096 L300.3552830256686 145.54160030464817 L310.91257585444924 154.4104110661228 L321.46986868322995 164.00429292701529 L332.02718668258217 174.17330976378256 L342.5844795113628 184.76214391619533 L353.14179751071504 195.61193471944458 L363.6990903394957 206.56152334358833 L374.2563831682764 217.4492338715298 L384.8137011676286 228.114443711858 L395.37099399640925 238.399019952056 L405.9283119957615 248.14914831491893 L416.4856048245421 257.216549270937 L427.0428976533228 265.46021123783294 L437.60021565267505 272.74765457138534 L448.1575084814557 278.9561668291875 L458.71480131023634 283.97421039679114 L469.27214448016014 287.70242793495225 L479.8294373089408 290.0545616456841 L490.3867301377215 290.9583821139983 L500.94402296650213 290.3564352115735 L511.5013157952828 288.20648257840526 L522.0586589652066 284.4820665884014 L532.6159517939873 279.17265398470306 L543.1732446227679 272.28375078794113 " fill="none" stroke-width="4.4" stroke="rgb(228,26,28)" stroke-opacity="1.0">
 
 
 
 <path d="M25.865392601084185 433.3219773327592 L25.865392601084185 433.3219773327592 L36.42269565540954 430.44545261107066 L46.9799987097349 333.9337636764956 L57.537302550640625 254.24530902478833 L68.09460481838562 265.8042752852865 L78.65190708613062 283.4243073589376 L89.20921250019707 272.53476787457794 L99.76651162162062 247.45350330488975 L110.32381703568706 224.23335988960937 L120.8811224497535 209.96623583654693 L131.43842157117706 206.33837150918745 L141.99572698524352 206.5094890547051 L152.55303239930996 206.67968733416964 L163.1103378133764 142.8337680854777 L173.66763064215706 137.30344435838305 L184.2249360562235 132.93375147751192 L194.78224147028993 126.13028256956403 L205.3395468843564 109.82583512621966 L215.89685229842283 92.03020408504176 L226.4541451272035 94.0424774753755 L237.01145054126994 110.2884174647125 L247.56876854062216 107.96629396034064 L258.12606136940286 93.83355511342228 L268.6833541981835 86.99955891195958 L279.24067219753573 98.9957702403608 L289.7979650263164 119.68714680307284 L300.3552830256686 129.58865721837498 L310.91257585444924 141.37303517565465 L321.46986868322995 132.42325239599887 L332.02718668258217 116.9464125197502 L342.5844795113628 126.20514529876704 L353.14179751071504 164.66499625133693 L363.6990903394957 178.05617466400915 L374.2563831682764 206.77699547617024 L384.8137011676286 263.8309653696428 L395.37099399640925 253.03190656170148 L405.9283119957615 230.8758997841286 L416.4856048245421 236.63042867131003 L427.0428976533228 273.6535625396953 L437.60021565267505 289.86529817130486 L448.1575084814557 298.0002377760403 L458.71480131023634 313.69618854633126 L469.27214448016014 318.35519169036553 L479.8294373089408 317.2449768417442 L490.3867301377215 310.5196934267539 L500.94402296650213 298.7261515360069 L511.5013157952828 295.02046559183594 L522.0586589652066 288.1397591840478 L532.6159517939873 285.17704130026686 L543.1732446227679 284.6866607393544 " fill="none" stroke-wi

Comparing with nonparametric attention pooling,
the region with large attention weights becomes sharper
in the learnable and parametric setting.


In [120]:
plotHeatmap(net.attentionWeights!!, 500, 700)


<rect fill="#0000ff" stroke="#ffffff" stroke-width="0.0" x="126.32256799942438" y="315.109090909091" height="11.890

## Summary

* Nadaraya-Watson kernel regression is an example of machine learning with attention mechanisms.
* The attention pooling of Nadaraya-Watson kernel regression is a weighted average of the training outputs. From the attention perspective, the attention weight is assigned to a value based on a function of a query and the key that is paired with the value.
* Attention pooling can be either nonparametric or parametric.


## Exercises

1. Increase the number of training examples. Can you learn  nonparametric Nadaraya-Watson kernel regression better?
1. What is the value of our learned $w$ in the parametric attention pooling experiment? Why does it make the weighted region sharper when visualizing the attention weights?
1. How can we add hyperparameters to nonparametric Nadaraya-Watson kernel regression to predict better?
1. Design another parametric attention pooling for the kernel regression of this section. Train this new model and visualize its attention weights.
